In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxin-database/peptide.csv
/kaggle/input/toxin-database/protein_test1002.csv
/kaggle/input/toxin-database/protein_train1002.csv
/kaggle/input/cobmineddb/combined_protein.csv
/kaggle/input/cobmineddb/combined_peptides.csv


In [2]:
import pandas as pd
# Load the dataset
file_path = '/kaggle/input/cobmineddb/combined_protein.csv'
protein_data = pd.read_csv(file_path)
protein_data

,name,sequence,label,sequence_length
0,>HPCL1_XENTR,MGKQNSKLRPEVLQDLRENTEFTDHELQEWYKGFLKDCPTGHLTVE...,0,193
1,>SIX4_HOTTS,DGYIKGNKGCKVSCVINNVFCNSMCKSSGGSYGYCWSWGLACWCEG...,1,62
2,>TAG2L_AGEOR,MRAIISLLLISAMVFSIIEAVPEEEGLQLSEDERGGCLPHNRFCNA...,1,70
3,>P2011_DANRE,MKTKFTKKTVLKFFGILFAILLLSVLILFSVVIGRTFTFKVNRELG...,0,515
4,>O165_CONTE,MKLTCMVIVAVLFLTAWTFVTAITSNGLENLFPNAHHEMKNPEASK...,1,76
...,...,...,...,...
9051,>EMC2B_XENLA,MSKVSDLYDVTWEDMRDKMKTWREDNYRNSEQIVDVGEELINEHAS...,0,297
9052,>COLI_LITCT,MLQPVWHACILAILGVFIFHVGEVRSQCWESNKCTDLSSEDGILEC...,0,263
9053,>MED30_DANRE,MTTPPLAQFSGQQQQQTQAARDVNTASLCRIGQETVQDIVLRTMEI...,0,174
9054,>RB87F_DROME,MAEQNDSNGNYDDGEEITEPEQLRKLFIGGLDYRTTDDGLKAHFEK...,0,385


In [3]:
# Load the dataset
file_path = '/kaggle/input/cobmineddb/combined_peptides.csv'
peptide_data = pd.read_csv(file_path)
peptide_data

,name,sequence,label,sequence_length
0,>COC2C_CONCL,DVCDSLVGGRCIHNGCYCERDAPNGNCCNTDGCTARWWCPGTKWD,1,45
1,>NA15_ANTXA,GVSCLCDSDGPSVSGNTLSGIIWLAGCPSGWHNCKAHGPNIGWCCKK,1,47
2,>PPK6_SHELA,SESEVPGMWFGPRL,0,14
3,>SK1_BLAGE,EQFDDYGHMRF,0,11
4,>COMB_CONMA,AATCTHWALIYFKTVQLFGWHFNYQVDATYCPQFQPCMP,1,39
...,...,...,...,...
5616,NaN,EDDHHHHHHHHHGVGGGGGGGGGG,0,24
5617,NaN,MTTNTQYIYPIFTVRWLAVHALAVPTVFFLGSISAMQFIQR,0,41
5618,NaN,GILDVAKTLVGKLRNVLGI,0,19
5619,NaN,MKVLSSLASAKTRYPDCQVVRRRGRVYVICKSNPRFKAVQGRKKRR,0,46


# Train Test Split

In [4]:
%%capture
!pip install lazypredict
import pandas as pd
import numpy as np
from collections import Counter  # Import Counter
import itertools
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import StandardScaler

# Protein Prediction

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
from collections import Counter
from lazypredict.Supervised import LazyClassifier

# Load the dataset
data = protein_data

# Define the feature extraction functions
def calculate_aac(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    sequence_length = len(sequence)
    aa_count = Counter(sequence)
    aac = [aa_count[aa] / sequence_length for aa in amino_acids]
    return aac

def calculate_dpc(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    dipeptides = [''.join(pair) for pair in itertools.product(amino_acids, repeat=2)]
    dipeptide_count = Counter([sequence[i:i+2] for i in range(len(sequence)-1)])
    sequence_length = len(sequence) - 1
    dpc = [dipeptide_count[dp] / sequence_length for dp in dipeptides]
    return dpc

# Extract features for a given dataset with renamed columns to avoid overlap
def extract_features(data):
    aac_features = data['sequence'].apply(calculate_aac)
    dpc_features = data['sequence'].apply(calculate_dpc) 

    aac_df = pd.DataFrame(aac_features.tolist(), columns=[f'aac_{i}' for i in range(20)])
    dpc_df = pd.DataFrame(dpc_features.tolist(), columns=[f'dpc_{i}' for i in range(400)])
  
    features = aac_df.join(dpc_df)
    
    return features

# Extract features and save them to CSV
features = extract_features(data)
features['label'] = data['label']
features.to_csv('/kaggle/working/protein_features.csv', index=False)



In [ ]:
# Load the features from the saved CSV
loaded_data = pd.read_csv('/kaggle/working/protein_features.csv')

# Split the data into training (60%), validation (20%), and testing (20%) sets
train_data, temp_data = train_test_split(loaded_data, test_size=0.4, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Separate features and labels
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']
X_val = val_data.drop(columns=['label'])
y_val = val_data['label']
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']




In [ ]:
# Evaluate models using LazyPredict
clf = LazyClassifier()
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Display model performance
print(models)

In [ ]:
loaded_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("full data")
# Calculate label distribution
label_distribution = loaded_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
train_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Training")
# Calculate label distribution
label_distribution = train_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
test_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Testing")
# Calculate label distribution
label_distribution = test_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
val_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Validation")
# Calculate label distribution
label_distribution = val_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
best_model = models.index[0]
best_model

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score, precision_recall_curve, auc, confusion_matrix, classification_report

# Define the model dictionary including AdaBoostClassifier
model_dict = {
    'LGBMClassifier': LGBMClassifier(),
    'XGBClassifier': XGBClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(probability=True),  # SVC needs probability=True for AUROC
    'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
}

# Function to calculate additional metrics
def calculate_metrics(y_true, y_pred, y_prob=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    sensitivity = tp / (tp + fn)  # Sensitivity (SN)
    specificity = tn / (tn + fp)  # Specificity (SP)
    fdr = fp / (fp + tp)  # False Discovery Rate (FDR)
    
    if y_prob is not None:
        auroc = roc_auc_score(y_true, y_prob[:, 1])
        precision, recall, _ = precision_recall_curve(y_true, y_prob[:, 1])
        auprc = auc(recall, precision)
    else:
        auroc = None
        auprc = None

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'mcc': mcc,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'fdr': fdr,
        'auroc': auroc,
        'auprc': auprc
    }

# Fit and evaluate each model
best_model_name = None
best_accuracy = 0
results = {}

for model_name, model in model_dict.items():
    # Fit the model
    model.fit(X_train, y_train)
    
    # Make predictions
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    
    # Probability predictions for ROC and AUPRC
    if hasattr(model, "predict_proba"):
        val_prob = model.predict_proba(X_val)
        test_prob = model.predict_proba(X_test)
    else:
        val_prob = None
        test_prob = None
    
    # Calculate metrics for validation set
    val_metrics = calculate_metrics(y_val, val_predictions, val_prob)
    test_metrics = calculate_metrics(y_test, test_predictions, test_prob)
    
    # Store results
    results[model_name] = {
        'validation_metrics': val_metrics,
        'test_metrics': test_metrics
    }
    
    # Print metrics
    print(f"\n{model_name} Validation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print(f"\n{model_name} Test Metrics:")
    for metric, value in test_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # Track the best model by accuracy
    if val_metrics['accuracy'] > best_accuracy:
        best_accuracy = val_metrics['accuracy']
        best_model_name = model_name

print(f"\nBest Model Name: {best_model_name}")

In [ ]:
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define base models for stacking
base_models = [
    ('lgbm', LGBMClassifier()),  # LightGBM model
    ('svc', SVC(probability=True)),  # Support Vector Classifier
    ('xgb', XGBClassifier(tree_method='gpu_hist', gpu_id=0))  # XGBoost model with GPU
]

# Create stacking classifier
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),  # Final estimator
    cv=5  # Use 5-fold cross-validation
)

# Fit the stacking model
stacking_model.fit(X_train, y_train)


In [ ]:
# Make predictions on validation and test sets
val_predictions = stacking_model.predict(X_val)
test_predictions = stacking_model.predict(X_test)

# Probability predictions for ROC and AUPRC
if hasattr(stacking_model, "predict_proba"):
    val_prob = stacking_model.predict_proba(X_val)
    test_prob = stacking_model.predict_proba(X_test)
else:
    val_prob = None
    test_prob = None

# Calculate metrics for validation set
val_metrics = calculate_metrics(y_val, val_predictions, val_prob)
test_metrics = calculate_metrics(y_test, test_predictions, test_prob)

# Print stacking model metrics
print("\nStacking Model Validation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nStacking Model Test Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Perform cross-validation for stacking model using the training set
n_folds = 5  # You can adjust this number
cv_scores = cross_val_score(stacking_model, X_train, y_train, cv=n_folds, scoring='accuracy')

# Calculate average cross-validation score
average_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

# Print cross-validation results
print("\nCross-Validation Scores for Stacking Model:")
print(f"Scores: {cv_scores}")
print(f"Average Score: {average_cv_score:.4f} ± {std_cv_score:.4f}")

# Peptide prediction

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
from collections import Counter
from lazypredict.Supervised import LazyClassifier

# Load the dataset
data = peptide_data

# Define the feature extraction functions
def calculate_aac(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    sequence_length = len(sequence)
    aa_count = Counter(sequence)
    aac = [aa_count[aa] / sequence_length for aa in amino_acids]
    return aac

def calculate_dpc(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    dipeptides = [''.join(pair) for pair in itertools.product(amino_acids, repeat=2)]
    dipeptide_count = Counter([sequence[i:i+2] for i in range(len(sequence)-1)])
    sequence_length = len(sequence) - 1
    dpc = [dipeptide_count[dp] / sequence_length for dp in dipeptides]
    return dpc

# Extract features for a given dataset with renamed columns to avoid overlap
def extract_features(data):
    aac_features = data['sequence'].apply(calculate_aac)
    dpc_features = data['sequence'].apply(calculate_dpc)

    aac_df = pd.DataFrame(aac_features.tolist(), columns=[f'aac_{i}' for i in range(20)])
    dpc_df = pd.DataFrame(dpc_features.tolist(), columns=[f'dpc_{i}' for i in range(400)])

    features = aac_df.join(dpc_df)
    
    return features

# Extract features and save them to CSV
features = extract_features(data)
features['label'] = data['label']
features.to_csv('/kaggle/working/peptide_features.csv', index=False)



In [ ]:
# Load the features from the saved CSV
loaded_data = pd.read_csv('/kaggle/working/peptide_features.csv')

# Split the data into training (60%), validation (20%), and testing (20%) sets
train_data, temp_data = train_test_split(loaded_data, test_size=0.4, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Separate features and labels
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']
X_val = val_data.drop(columns=['label'])
y_val = val_data['label']
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']

# Evaluate models using LazyPredict
clf = LazyClassifier()
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Display model performance
print(models)


In [ ]:
best_model = models.index[0]
best_model

In [ ]:
loaded_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("full")
# Calculate label distribution
label_distribution = loaded_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
train_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Training")
# Calculate label distribution
label_distribution = train_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
test_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Test")
# Calculate label distribution
label_distribution = test_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
val_data

In [ ]:
#train_data,val_data,test_data 

import matplotlib.pyplot as plt
print("Validation")
# Calculate label distribution
label_distribution = val_data['label'].value_counts()
print(label_distribution)
# Calculate the percentages of each label
label_percentages = (label_distribution / label_distribution.sum()) * 100
print(label_percentages)


In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score, precision_recall_curve, auc, confusion_matrix, classification_report

# Define the model dictionary including AdaBoostClassifier
model_dict = {
    'LGBMClassifier': LGBMClassifier(),
    'XGBClassifier': XGBClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(probability=True),  # SVC needs probability=True for AUROC
    'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
}

# Function to calculate additional metrics
def calculate_metrics(y_true, y_pred, y_prob=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    sensitivity = tp / (tp + fn)  # Sensitivity (SN)
    specificity = tn / (tn + fp)  # Specificity (SP)
    fdr = fp / (fp + tp)  # False Discovery Rate (FDR)
    
    if y_prob is not None:
        auroc = roc_auc_score(y_true, y_prob[:, 1])
        precision, recall, _ = precision_recall_curve(y_true, y_prob[:, 1])
        auprc = auc(recall, precision)
    else:
        auroc = None
        auprc = None

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'mcc': mcc,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'fdr': fdr,
        'auroc': auroc,
        'auprc': auprc
    }

# Fit and evaluate each model
best_model_name = None
best_accuracy = 0
results = {}

for model_name, model in model_dict.items():
    # Fit the model
    model.fit(X_train, y_train)
    
    # Make predictions
    val_predictions = model.predict(X_val)
    test_predictions = model.predict(X_test)
    
    # Probability predictions for ROC and AUPRC
    if hasattr(model, "predict_proba"):
        val_prob = model.predict_proba(X_val)
        test_prob = model.predict_proba(X_test)
    else:
        val_prob = None
        test_prob = None
    
    # Calculate metrics for validation set
    val_metrics = calculate_metrics(y_val, val_predictions, val_prob)
    test_metrics = calculate_metrics(y_test, test_predictions, test_prob)
    
    # Store results
    results[model_name] = {
        'validation_metrics': val_metrics,
        'test_metrics': test_metrics
    }
    
    # Print metrics
    print(f"\n{model_name} Validation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print(f"\n{model_name} Test Metrics:")
    for metric, value in test_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    # Track the best model by accuracy
    if val_metrics['accuracy'] > best_accuracy:
        best_accuracy = val_metrics['accuracy']
        best_model_name = model_name

print(f"\nBest Model Name: {best_model_name}")

In [ ]:
from sklearn.ensemble import StackingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

# Define base models for stacking
base_models = [
    ('lgbm', LGBMClassifier()),  # LightGBM model
    ('extra_trees', ExtraTreesClassifier()),  # Extra Trees Classifier
    ('xgb', XGBClassifier(tree_method='gpu_hist', gpu_id=0))  # XGBoost model with GPU
]

# Create stacking classifier
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),  # You can choose any classifier here
    cv=5  # Use 5-fold cross-validation
)

# Fit the stacking model
stacking_model.fit(X_train,y_train)

In [ ]:
# Make predictions on validation and test sets
val_predictions = stacking_model.predict(X_val)
test_predictions = stacking_model.predict(X_test)

# Probability predictions for ROC and AUPRC
if hasattr(stacking_model, "predict_proba"):
    val_prob = stacking_model.predict_proba(X_val)
    test_prob = stacking_model.predict_proba(X_test)
else:
    val_prob = None
    test_prob = None

# Calculate metrics for validation set
val_metrics = calculate_metrics(y_val, val_predictions, val_prob)
test_metrics = calculate_metrics(y_test, test_predictions, test_prob)

# Print stacking model metrics
print("\nStacking Model Validation Metrics:")
for metric, value in val_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nStacking Model Test Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Perform cross-validation for stacking model using the training set
n_folds = 5  # You can adjust this number
cv_scores = cross_val_score(stacking_model, X_train, y_train, cv=n_folds, scoring='accuracy')

# Calculate average cross-validation score
average_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

# Print cross-validation results
print("\nCross-Validation Scores for Stacking Model:")
print(f"Scores: {cv_scores}")
print(f"Average Score: {average_cv_score:.4f} ± {std_cv_score:.4f}")